In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install ctgan

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.7/66.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report
from imblearn.over_sampling import RandomOverSampler, SMOTE
from imblearn.under_sampling import RandomUnderSampler
from ctgan import CTGAN
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/프로젝트 1/MBA.csv')

In [5]:

# 'race' 컬럼에서 결측치를 'unknown'으로 채움
df['race'] = df['race'].fillna('unknown')

# 'admission' 컬럼에서 결측치를 'denied'로 채움
df['admission'] = df['admission'].fillna('denied')

# 'admission' 컬럼에서 'Waitlist' 값을 'denied'로 변환
df['admission'] = df['admission'].apply(lambda x : 'Admit' if x == 'Waitlist' else x)

# 'admission' 컬럼을 이진 변수로 변환 ('denied' = 1, 그 외 = 0)
df['admission'] = df['admission'].apply(lambda x : 1 if x == 'denied' else 0)

# 'application_id' 컬럼은 필요 없으므로 제거
df = df.drop('application_id', axis = 1)

# 데이터 준비 (admission 데이터 준비 완료)
# X와 y는 이미 준비되어 있고, 소수 클래스(0)에 대해 증강하려고 함

# 소수 클래스만 추출 (여기서는 '0'이 소수 클래스라고 가정)
minority_class_data = df[df['admission'] == 0]  # admission이 0인 데이터 추출
minority_class_data = minority_class_data.drop(columns=['admission'])  # 타겟 열 제외

# CTGAN 모델 정의 및 학습
ctgan = CTGAN(epochs=100)  # 학습할 epoch 수는 필요에 따라 조정 가능
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()
ctgan.fit(minority_class_data, discrete_columns=categorical_cols)  # 범주형 컬럼 명시

# 새로운 데이터 생성 (예: 4800개의 소수 클래스 데이터 생성)
new_samples = ctgan.sample(4800)

# 생성된 데이터를 원래 소수 클래스의 레이블인 0으로 지정
new_samples['admission'] = 0

# 기존 데이터와 결합
df_augmented = pd.concat([df, new_samples], ignore_index=True)

# LabelEncoder 객체 생성
le = LabelEncoder()

# 범주형 컬럼(object 타입)을 모두 찾아서 레이블 인코딩 수행
categorical_cols = df_augmented.select_dtypes(include=['object']).columns

# 각 범주형 컬럼에 Label Encoding 적용
for col in categorical_cols:
    df_augmented[col] = le.fit_transform(df_augmented[col])  # 인코딩 적용
    print(f"Classes (범주) for {col}: {le.classes_}")  # 원래 범주들 출력
    encoded_data = df_augmented[col]  # 인코딩된 값을 데이터프레임에 저장
    mapping = dict(zip(le.classes_, range(len(le.classes_))))  # 범주와 인코딩 값 매핑
    print(f"Mapping (범주 -> 인코딩 값) for {col}: {mapping}\n")

Classes (범주) for gender: ['Female' 'Male']
Mapping (범주 -> 인코딩 값) for gender: {'Female': 0, 'Male': 1}

Classes (범주) for major: ['Business' 'Humanities' 'STEM']
Mapping (범주 -> 인코딩 값) for major: {'Business': 0, 'Humanities': 1, 'STEM': 2}

Classes (범주) for race: ['Asian' 'Black' 'Hispanic' 'Other' 'White' 'unknown']
Mapping (범주 -> 인코딩 값) for race: {'Asian': 0, 'Black': 1, 'Hispanic': 2, 'Other': 3, 'White': 4, 'unknown': 5}

Classes (범주) for work_industry: ['CPG' 'Consulting' 'Energy' 'Financial Services' 'Health Care'
 'Investment Banking' 'Investment Management' 'Media/Entertainment'
 'Nonprofit/Gov' 'Other' 'PE/VC' 'Real Estate' 'Retail' 'Technology']
Mapping (범주 -> 인코딩 값) for work_industry: {'CPG': 0, 'Consulting': 1, 'Energy': 2, 'Financial Services': 3, 'Health Care': 4, 'Investment Banking': 5, 'Investment Management': 6, 'Media/Entertainment': 7, 'Nonprofit/Gov': 8, 'Other': 9, 'PE/VC': 10, 'Real Estate': 11, 'Retail': 12, 'Technology': 13}



In [ ]:
X = df_augmented.drop(['admission', 'major', 'gender', 'race', 'international', 'work_industry'], axis=1)
# X = df_augmented.drop('admission', axis=1)
y = df_augmented['admission']

# 데이터 스케일링
scaler = StandardScaler()
X_scaled_augmented = scaler.fit_transform(X)

# 학습 및 테스트 데이터로 분리
X_train, X_test, y_train, y_test = train_test_split(X_scaled_augmented, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [ ]:
models = [
    ("Logistic Regression", LogisticRegression(class_weight='balanced')),
    ("K-Nearest Neighbors", KNeighborsClassifier(n_neighbors=3)),
    ("Decision Tree", DecisionTreeClassifier(class_weight='balanced', random_state=42)),
    ("Random Forest", RandomForestClassifier(class_weight='balanced', random_state=42)),
    ("Support Vector Machine", SVC(probability=True)),
    ("XGBOOST", XGBClassifier(scale_pos_weight=10, random_state=42)),
    ("GaussianNB", GaussianNB()),
    ("GradientBoostingClassifier", GradientBoostingClassifier()),
    ("AdaBoostClassifier", AdaBoostClassifier())
]

for model_name, model in models:
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    print(f"\nModel: {model_name}")
    print(f"Accuracy: {accuracy * 100:.2f}%")

    print("Classification Report:")
    print(classification_report(y_test, y_pred))

    if hasattr(model, "predict_proba"):
        y_probs = model.predict_proba(X_test)[:, 1]
        roc_auc = roc_auc_score(y_test, y_probs)
        print(f"ROC-AUC: {roc_auc:.2f}")


Model: Logistic Regression
Accuracy: 86.68%
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.86      0.87      1152
           1       0.85      0.88      0.86      1047

    accuracy                           0.87      2199
   macro avg       0.87      0.87      0.87      2199
weighted avg       0.87      0.87      0.87      2199

ROC-AUC: 0.94

Model: K-Nearest Neighbors
Accuracy: 85.13%
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.83      0.85      1152
           1       0.82      0.87      0.85      1047

    accuracy                           0.85      2199
   macro avg       0.85      0.85      0.85      2199
weighted avg       0.85      0.85      0.85      2199

ROC-AUC: 0.90

Model: Decision Tree
Accuracy: 85.77%
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.86      0.86      1152
           

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(



Model: AdaBoostClassifier
Accuracy: 87.99%
Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.87      0.88      1152
           1       0.86      0.89      0.88      1047

    accuracy                           0.88      2199
   macro avg       0.88      0.88      0.88      2199
weighted avg       0.88      0.88      0.88      2199

ROC-AUC: 0.96


In [ ]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier
import numpy as np

# 1. XGBoost 하이퍼파라미터 튜닝
xgb_model = XGBClassifier(scale_pos_weight=10, random_state=42)

# 하이퍼파라미터 탐색 공간 정의
xgb_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2]
}

# 2. GradientBoostingClassifier 하이퍼파라미터 튜닝
gbc_model = GradientBoostingClassifier(random_state=42)

# 하이퍼파라미터 탐색 공간 정의
gbc_param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'min_samples_split': [2, 5, 10]
}

# GridSearchCV를 통한 하이퍼파라미터 탐색
xgb_grid_search = GridSearchCV(xgb_model, param_grid=xgb_param_grid,
                               cv=3, verbose=1, n_jobs=-1)

gbc_grid_search = GridSearchCV(gbc_model, param_grid=gbc_param_grid,
                               cv=3, verbose=1, n_jobs=-1)

# XGBoost Grid Search
xgb_grid_search.fit(X_train, y_train)
print("Best XGBoost Parameters from Grid Search:", xgb_grid_search.best_params_)

# GradientBoosting Grid Search
gbc_grid_search.fit(X_train, y_train)
print("Best GradientBoosting Parameters from Grid Search:", gbc_grid_search.best_params_)

# 최종 모델 평가
xgb_best_model = xgb_grid_search.best_estimator_
gbc_best_model = gbc_grid_search.best_estimator_

xgb_y_pred = xgb_best_model.predict(X_test)
gbc_y_pred = gbc_best_model.predict(X_test)

# 성능 평가
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

print("\nXGBoost 최종 성능:")
print(f"Accuracy: {accuracy_score(y_test, xgb_y_pred) * 100:.2f}%")
print(classification_report(y_test, xgb_y_pred))
if hasattr(xgb_best_model, "predict_proba"):
    xgb_y_probs = xgb_best_model.predict_proba(X_test)[:, 1]
    xgb_roc_auc = roc_auc_score(y_test, xgb_y_probs)
    print(f"ROC-AUC: {xgb_roc_auc:.2f}")

print("\nGradientBoostingClassifier 최종 성능:")
print(f"Accuracy: {accuracy_score(y_test, gbc_y_pred) * 100:.2f}%")
print(classification_report(y_test, gbc_y_pred))
if hasattr(gbc_best_model, "predict_proba"):
    gbc_y_probs = gbc_best_model.predict_proba(X_test)[:, 1]
    gbc_roc_auc = roc_auc_score(y_test, gbc_y_probs)
    print(f"ROC-AUC: {gbc_roc_auc:.2f}")


In [ ]:
import numpy as np
from xgboost import XGBClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score

X = df_augmented[['gpa', 'gmat', 'work_exp']]
y = df_augmented['admission']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

xgb_model = XGBClassifier(scale_pos_weight=10, random_state=42)

xgb_model.fit(X_train_scaled, y_train)

def user_input():
    gpa = float(input("GPA 점수를 입력하세요 (예: 3.5): "))
    gmat = int(input("GMAT 점수를 입력하세요 (예: 700): "))
    work_exp = int(input("경력 년수를 입력하세요 (예: 5): "))

    user_data = np.array([[gpa, gmat, work_exp]])

    user_data_scaled = scaler.transform(user_data)

    return user_data_scaled

def predict_admission(model):
    user_data = user_input()

    prediction = model.predict(user_data)
    prediction_proba = model.predict_proba(user_data)[:, 1]

    if prediction == 1:
        print(f"\n입학 가능성이 높습니다!")
    else:
        print(f"\n입학 가능성이 낮습니다.")

predict_admission(xgb_model)


GPA 점수를 입력하세요 (예: 3.5): 2.7
GMAT 점수를 입력하세요 (예: 700): 400
경력 년수를 입력하세요 (예: 5): 1

입학 가능성이 낮습니다.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
